In [4]:
%cd C:\Users\Academic\Documents\Projects\sales-performance-dashboard

import pandas as pd
from sqlalchemy import create_engine

df_cleaned = pd.read_csv("data/processed/superstore_cleaned.csv", parse_dates=["Order Date", "Ship Date"])

engine = create_engine("sqlite:///data/superstore.db", echo=True)

df_cleaned.to_sql("raw_sales", engine, if_exists="replace", index=False)


C:\Users\Academic\Documents\Projects\sales-performance-dashboard
2025-05-21 11:57:00,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 11:57:00,602 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("raw_sales")
2025-05-21 11:57:00,603 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 11:57:00,605 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("raw_sales")
2025-05-21 11:57:00,607 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 11:57:00,613 INFO sqlalchemy.engine.Engine 
CREATE TABLE raw_sales (
	"Order ID" TEXT, 
	"Order Date" DATETIME, 
	"Ship Date" DATETIME, 
	"Country" TEXT, 
	"Region" TEXT, 
	"Product ID" TEXT, 
	"Category" TEXT, 
	"Sub-Category" TEXT, 
	"Product Name" TEXT, 
	"Sales" FLOAT, 
	"Quantity" BIGINT, 
	"Profit" FLOAT
)


2025-05-21 11:57:00,614 INFO sqlalchemy.engine.Engine [no key 0.00134s] ()
2025-05-21 11:57:00,887 INFO sqlalchemy.engine.Engine INSERT INTO raw_sales ("Order ID", "Order Date", "Ship Date", "Country", "Region", "Product ID

9994

In [9]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text(
    
"""
CREATE TABLE IF NOT EXISTS dim_date AS
SELECT 
   DISTINCT DATE("Order Date") AS order_date,
   CAST(STRFTIME('%Y', "Order Date") AS INTEGER) AS year,
   CAST(STRFTIME('%m', "Order Date") AS INTEGER) AS month,

   CASE 
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 7 AND 8 THEN 'Q3'
       ELSE 'Q4'

  END                                             AS quarter

FROM raw_sales; """))

with engine.begin() as conn:
    conn.execute(text(
"""
CREATE TABLE IF NOT EXISTS dim_product AS
    SELECT DISTINCT
      Category,
      "Sub-Category" AS sub_category

FROM raw_sales
   """         
                   )
                )

with engine.begin() as conn:
    conn.execute(text(
"""
CREATE TABLE IF NOT EXISTS dim_region AS
    SELECT DISTINCT
      Region
      
FROM raw_sales
   """         
                   )
                )






2025-05-21 12:19:04,581 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 12:19:04,582 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS dim_date AS
SELECT 
   DISTINCT DATE("Order Date") AS order_date,
   CAST(STRFTIME('%Y', "Order Date") AS INTEGER) AS year,
   CAST(STRFTIME('%m', "Order Date") AS INTEGER) AS month,

   CASE 
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2'
       WHEN CAST(STRFTIME('%m', "Order Date") AS INTEGER) BETWEEN 7 AND 8 THEN 'Q3'
       ELSE 'Q4'

  END                                             AS quarter

FROM raw_sales; 
2025-05-21 12:19:04,582 INFO sqlalchemy.engine.Engine [generated in 0.00050s] ()
2025-05-21 12:19:04,599 INFO sqlalchemy.engine.Engine COMMIT
2025-05-21 12:19:04,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 12:19:04,601 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS dim_produc

In [10]:


with engine.begin() as conn:

    conn.execute(text
       
    ("""
        CREATE TABLE IF NOT EXISTS facts_sales AS
        SELECT 
            ROW_NUMBER() OVER (ORDER by "ORDER ID") AS sale_id,
            DATE("Order Date")                      AS order_date,
            Category,
            "Sub-Category" AS sub_category,
            Region,
            Sales,
            Quantity,
            Profit,
            CAST((JULIANDAY("Ship Date") - JULIANDAY("Order Date")) AS INTEGER) AS shipping_time_days
    FROM raw_sales;


    
      """)
            )

2025-05-21 12:19:08,698 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 12:19:08,698 INFO sqlalchemy.engine.Engine 
        CREATE TABLE IF NOT EXISTS facts_sales AS
        SELECT 
            ROW_NUMBER() OVER (ORDER by "ORDER ID") AS sale_id,
            DATE("Order Date")                      AS order_date,
            Category,
            "Sub-Category" AS sub_category,
            Region,
            Sales,
            Quantity,
            Profit,
            CAST((JULIANDAY("Ship Date") - JULIANDAY("Order Date")) AS INTEGER) AS shipping_time_days
    FROM raw_sales;



      
2025-05-21 12:19:08,699 INFO sqlalchemy.engine.Engine [cached since 1202s ago] ()
2025-05-21 12:19:08,699 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
with engine.begin() as conn:

    conn.execute(text(
"""

CREATE VIEW IF NOT EXISTS top3_products_per_subcategory AS
SELECT
    sub_category,
    product_name,
    total_sales
FROM (
    SELECT
        "Sub-Category" AS sub_category,
        "Product Name" AS product_name,
        SUM(Sales) AS total_sales,
        ROWNUMBER() OVER (
         PARTITION  BY "Sub-Category"
         ORDER BY SUM(SALES) DESC
        )   AS sales_rank
FROM raw_sales
GROUP BY "Sub-Category", "Product Name"
)
WHERE sales_rank <=3

"""    
))

    conn.execute(text(
    """ SELECT * FROM top3_products_per_subcategory;"""

        
    ))



2025-05-21 12:33:54,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 12:33:54,537 INFO sqlalchemy.engine.Engine 

CREATE VIEW IF NOT EXISTS top3_products_per_subcategory AS
SELECT
    sub_category,
    product_name,
    total_sales
FROM (
    SELECT
        "Sub-Category" AS sub_category,
        "Product Name" AS product_name,
        SUM(Sales) AS total_sales,
        ROWNUMBER() OVER (
         PARTITION  BY "Sub-Category"
         ORDER BY SUM(SALES) DESC
        )   AS sales_rank
FROM raw_sales
GROUP BY "Sub-Category", "Product Name"
)
WHERE sales_rank <=3


2025-05-21 12:33:54,538 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ()
2025-05-21 12:33:54,538 INFO sqlalchemy.engine.Engine  SELECT * FROM top3_products_per_subcategory;
2025-05-21 12:33:54,540 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ()
2025-05-21 12:33:54,540 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such function: ROWNUMBER
[SQL:  SELECT * FROM top3_products_per_subcategory;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [17]:
pd.read_sql("SELECT Region, SUM(SALES) AS total_sales FROM facts_sales GROUP BY REGION", engine)

pd.read_sql("SELECT d.quarter, COUNT(*) AS num_orders FROM facts_sales f JOIN dim_date d USING(order_date) GROUP BY d.quarter", engine)

pd.read_sql("""SELECT sub_category, SUM(Profit) AS total_profit
            FROM facts_sales
            GROUP BY sub_category
            ORDER BY total_profit DESC LIMIT 5""", engine)




2025-05-21 12:31:01,967 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 12:31:01,967 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT Region, SUM(SALES) AS total_sales FROM facts_sales GROUP BY REGION")
2025-05-21 12:31:01,968 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 12:31:01,968 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT Region, SUM(SALES) AS total_sales FROM facts_sales GROUP BY REGION")
2025-05-21 12:31:01,969 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 12:31:01,970 INFO sqlalchemy.engine.Engine SELECT Region, SUM(SALES) AS total_sales FROM facts_sales GROUP BY REGION
2025-05-21 12:31:01,970 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 12:31:01,973 INFO sqlalchemy.engine.Engine ROLLBACK
2025-05-21 12:31:01,974 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 12:31:01,974 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT d.quarter, COUNT(*) AS num_orders FROM facts_sales f JOIN dim_date d USIN

,sub_category,total_profit
0,Copiers,55617.8249
1,Phones,44515.7306
2,Accessories,41936.6357
3,Paper,34053.5693
4,Binders,30221.7633
